In [1]:
# import statements
import torch
from sys import float_info
import sys
import pandas as pd

In [2]:
dataset1 = pd.read_csv('train.csv')
dataset2 = pd.read_csv('test.csv')

np1 = dataset1.values
train = torch.from_numpy(np1)

np2 = dataset2.values
test = torch.from_numpy(np2)

In [3]:
trainX = train[...,0:train.size(dim=1)-1]
trainY = train[...,train.size(dim=1)-1]
testX = test[...,0:test.size(dim=1)-1]
testY = test[...,test.size(dim=1)-1]

trainX = trainX.to(torch.float64)
trainY = trainY.to(torch.float64)
testX = testX.to(torch.float64)
testY = testY.to(torch.float64)

trainY = trainY.unsqueeze(1)
testY = testY.unsqueeze(1)

In [4]:
trainX.size()
trainX.size(dim=0)
trainX.size(dim=1)

10

In [5]:
#q1
trainX.size(dim=0) * trainX.size(dim=1) * 8

83520

In [6]:
#q2
trainX16 = trainX.to(torch.float16)
trainX16 = trainX16.to(torch.float64)
difference = trainX - trainX16
difference = torch.max(difference)
x = difference.clone().detach()

x.item()

0.0

In [7]:
#q3
torch.cuda.is_available()

False

In [8]:
coef = torch.tensor([
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0040], # POS_50_59_CP
        [0.0300], # POS_60_69_CP
        [0.0300],
        [0.0300],
        [0.0300]
], dtype=trainX.dtype)
coef

tensor([[0.0040],
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0300],
        [0.0300],
        [0.0300],
        [0.0300]], dtype=torch.float64)

In [9]:
#q4
firstCensus = testX[0] @ coef
firstCensus.item()

9.844

In [10]:
#q5
mult = testX @ coef
torch.mean(mult).item()

12.073632183908048

In [11]:
#q6
x = torch.tensor(0.0)
def f(x):
    return x ** 2 - 8*x + 19
y = f(x)
float(y)

19.0

In [12]:
#q7
# want to find x value that minimizes y
x = torch.tensor(0.0, requires_grad=True)
optimizer = torch.optim.SGD([x], lr = 0.1)

for epoch in range(100):
    y = f(x)
    y.backward()
    optimizer.step()
    optimizer.zero_grad()
float(x)

3.999999523162842

In [13]:
coef = torch.zeros(10, 1, dtype=float)
zeroes = trainX @ coef

In [14]:
#q8
mse = ((zeroes - trainY)**2).mean()
float(mse)

197.8007662835249

In [15]:
coef2 = torch.zeros((10, 1), dtype=torch.float64, requires_grad=True)
loss_fn = torch.nn.MSELoss()

torch.manual_seed(544)
ds = torch.utils.data.TensorDataset(trainX, trainY)
dl = torch.utils.data.DataLoader(ds, batch_size=50, shuffle=True)
optimizer = torch.optim.SGD([coef2], lr=0.000002)

for epoch in range(500):
    for batchX, batchY in dl:
        predictions = batchX @ coef2
        loss = loss_fn(predictions, batchY)
        loss.backward()   # computes gradient, and adds it to coef.grad
        optimizer.step()
        optimizer.zero_grad()

In [16]:
#q9
float(loss_fn(trainX @ coef2, trainY))

26.8113940147193

In [17]:
#q10
float(loss_fn(testX @ coef2, testY))

29.05854692548551